# Report_3
**Made by :**

Mohamed Ehab Yousri 202201236

Hamza Abdelmoreed   202201508

Amin Gamal          202202219

# Part 1 :1D minimization algorithms:

### Fibonacci Method

In [ ]:
import math

def fibonacci_method(f, a, b, tol):

    fib = [1, 1]
    while fib[-1] < (b - a) / tol:
        fib.append(fib[-1] + fib[-2])
    n = len(fib) - 1

    c = a + fib[n-2]/fib[n]* (b - a)
    d = a + fib[n-1]/fib[n]* (b - a)
    fc = f(c)
    fd = f(d)

    for k in range(1, n):
        if fc < fd:
            b = d
            d = c
            fd = fc
            c = a + fib[n - k - 2]/fib[n - k] * (b - a)
            fc = f(c)
        else:
            a = c
            c = d
            fc = fd
            d = a + fib[n - k -1]/fib[n -k] * (b - a)
            fd = f(d)
    return (b + a)/2


In [ ]:
def test_function(x):
    return (x - 2)**2

# interval
a = 0
b = 5
tolerance = 1e-5

minimum = fibonacci_method(test_function, a, b, tolerance)
print(f"minimum is at x = {minimum}")
print(f"value at minimum: f({minimum}) = {test_function(minimum)}")


minimum is at x = 2.000008750965037
value at minimum: f(2.000008750965037) = 7.657938907970796e-11


### **Golden Section Method**

In [ ]:
import math

def golden_section_method(f, a, b, tol=1e-5):
    gr = (math.sqrt(5) + 1) / 2
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c
        c = b - (b - a) / gr
        d = a + (b - a) / gr
    return (b + a) / 2


In [ ]:
def test_function1(x):
    return math.sin(x) + (x - 3)**2

# interval
a = 0
b = 6
tolerance = 1e-5

minimum = golden_section_method(test_function1, a, b, tolerance)
print(f"minimum is at x = {minimum}")
print(f"value at minimum: f({minimum}) = {test_function1(minimum)}")


minimum is at x = 3.472819998379147
value at minimum: f(3.472819998379147) = -0.10164515331177024


### **Newton’s Method**

In [ ]:
def newtons_method(f, df, ddf, x0, tol=1e-5, max_iter=100):
    x = x0
    for _ in range(max_iter):
        grad = df(x)
        hess = ddf(x)
        if hess == 0:
            return("Zero second derivative, No solution found")
        x_new = x - grad / hess
        if abs(x_new - x) < tol:
            return x_new
        x = x_new
    return("Maximum iterations exceeded")


In [ ]:
def test_function2(x):
    return x**3 - 3*x**2 + 4

def test_function_derivative(x):
    return 3*x**2 - 6*x

def test_function_second_derivative(x):
    return 6*x - 6

# initial guess
x0 = 0.0
tolerance = 1e-5

minimum = newtons_method(test_function2, test_function_derivative, test_function_second_derivative, x0, tolerance)
print(f"minimum is at x = {minimum}")
print(f"value at minimum: f({minimum}) = {test_function2(minimum)}")

minimum is at x = 0.0
value at minimum: f(0.0) = 4.0


### **Quasi-Newton Method**

In [ ]:
import math

def quasi_newton_method_1d(f, grad_f, x0, tol=1e-5, max_iter=100):

    x = x0
    H = 1.0  # Initial approximation

    for i in range(max_iter):
        grad = grad_f(x)
        if abs(grad) < tol:
            print(f"Converged in {i+1} iterations.")
            return x
        p = -H * grad

        # Line search
        def phi(alpha):
            return f(x + alpha * p)

        alpha = golden_section_method(phi, 0, 1, tol)

        x_new = x + alpha * p
        grad_new = grad_f(x_new)
        s = x_new - x
        y = grad_new - grad + 1*10**13

        if abs(x_new - x) < tol:
            print(f"Converged in {i+1} iterations")
            return x_new

        x = x_new

    return("Maximum iterations exceeded")


In [ ]:
import math

def test_function3(x):
    return math.exp(x) - 4*x

def test_function_derivative3(x):
    return math.exp(x) - 4

# Initial guess
x0 = 0.0
tolerance = 1e-5

minimum = quasi_newton_method_1d(test_function3, test_function_derivative3, x0, tolerance)
print(f"minimum is at x = {minimum}")
print(f"value at minimum: f({minimum}) = {test_function3(minimum)}")



Converged in 2 iterations
minimum is at x = 1.386294378858634
value at minimum: f(1.386294378858634) = -1.5451774444795623


### **Secant Method**

In [ ]:
def secant_method(f, a, b, tol=1e-5, max_iter=100):
    x0, x1 = a, b
    f0, f1 = f(x0), f(x1)
    for _ in range(max_iter):
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0 + 1*10**13)
        if abs(x2 - x1) < tol:
            return x2
        x0, f0 = x1, f1
        x1, f1 = x2, f(x2)

    return("maximum iterations exceeded")


In [ ]:
def test_function4(x):
    return (x - 1) * (x - 3)

# Initial guesses
a = 0.0
b = 4.0
tolerance = 1e-5

minimum = secant_method(test_function4, a, b, tolerance)
print(f"minimum is at x = {minimum}")
print(f"value at minimum: f({minimum}) = {test_function4(minimum)}")



minimum is at x = 3.9999999999988
value at minimum: f(3.9999999999988) = 2.9999999999952003


# Part 2 :unconstrained nonlinear optimization problem algorithms :

In [ ]:
import numpy as np

### Fletcher-Reeves Conjugate Gradient Method

In [ ]:
def fletcher_reeves_cg(f, grad_f, x0, tol=1e-5, max_iter=1000):
    x = np.array(x0, dtype=float)
    grad = grad_f(x)
    d = -grad
    history = {"iterations": [], "function_values": [], "gradient_norms": []}

    for iteration in range(max_iter):
        grad_norm = np.linalg.norm(grad)
        history["iterations"].append(iteration)
        history["function_values"].append(f(x))
        history["gradient_norms"].append(grad_norm)

        if grad_norm < tol:
            return x, history

        # Line search
        alpha = golden_section_method(lambda a: f(x + a * d), 0, 1)

        # Update x, gradient, and search direction
        x_new = x + alpha * d
        grad_new = grad_f(x_new)
        beta = np.dot(grad_new, grad_new) / np.dot(grad, grad)
        d = -grad_new + beta * d

        # Update variables for the next iteration
        x, grad = x_new, grad_new

    return x, history


Example Usage :

In [ ]:
# Define a test function and its gradient
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

def grad_rosenbrock(x):
    grad = np.zeros_like(x)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[1] = 200 * (x[1] - x[0]**2)
    return grad

# Initial guess
x0 = [-1.2, 1.0]

optimal_x, history = fletcher_reeves_cg(rosenbrock, grad_rosenbrock, x0)
print("Optimal solution:", optimal_x)
print("Function value at optimum:", rosenbrock(optimal_x))


Optimal solution: [1.00000291 1.00000585]
Function value at optimum: 8.563102755875322e-12


## Marquardt Method

In [ ]:
def marquardt_method(f, grad_f, hessian_f, x0, tol=1e-5, max_iter=100, lambda_init=0.01):

    x = np.array(x0, dtype=float)
    lambda_param = lambda_init
    history = {"iterations": [], "function_values": [], "gradient_norms": []}

    for iteration in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)

        history["iterations"].append(iteration)
        history["function_values"].append(f(x))
        history["gradient_norms"].append(grad_norm)

        # Check convergence
        if grad_norm < tol:
            return x, history

        # Compute Hessian and regularized version
        hessian = hessian_f(x)
        hessian_reg = hessian + lambda_param * np.eye(len(x))

        # Compute step direction
        try:
            delta = np.linalg.solve(hessian_reg, -grad)
        except np.linalg.LinAlgError:
            lambda_param *= 10
            continue

        # Update position
        x_new = x + delta


        if f(x_new) < f(x):
            x = x_new
            lambda_param /= 10
        else:
            lambda_param *= 10

        if np.linalg.norm(delta) < tol:
            return x, history

    return x, history


Example Usage

In [ ]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2


def grad_rosenbrock(x):
    grad = np.zeros_like(x)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[1] = 200 * (x[1] - x[0]**2)
    return grad

# Define the Hessian
def hessian_rosenbrock(x):
    hessian = np.zeros((len(x), len(x)))
    hessian[0, 0] = 1200 * x[0]**2 - 400 * x[1] + 2
    hessian[0, 1] = -400 * x[0]
    hessian[1, 0] = -400 * x[0]
    hessian[1, 1] = 200
    return hessian

# Initial guess
x0 = [-1.2, 1.0]


x_opt, history = marquardt_method(
    f=rosenbrock,
    grad_f=grad_rosenbrock,
    hessian_f=hessian_rosenbrock,
    x0=x0,
    tol=1e-5,
    max_iter=100,
    lambda_init=0.01
)


print("\nMarquardt Method Results:")
print(f"Optimal solution: {x_opt}")
print(f"Function value at optimum: {rosenbrock(x_opt)}")
print(f"Number of iterations: {len(history['iterations'])}")



Marquardt Method Results:
Optimal solution: [0.99999998 0.99999996]
Function value at optimum: 4.866187419097994e-16
Number of iterations: 41


## Quasi-Newton Method

In [ ]:
def quasi_newton_bfgs(f, grad_f, x0, tol=1e-5, max_iter=100):
    x = np.array(x0, dtype=float)
    n = len(x)
    H = np.eye(n)
    grad = grad_f(x)
    history = {"iterations": [], "function_values": [], "gradient_norms": []}

    for iteration in range(max_iter):
        grad_norm = np.linalg.norm(grad)
        history["iterations"].append(iteration)
        history["function_values"].append(f(x))
        history["gradient_norms"].append(grad_norm)

        # Convergence check
        if grad_norm < tol:
            return x, history

        # Compute search direction
        d = -np.dot(H, grad)

        # Line search to find step size
        alpha = golden_section_method(lambda a: f(x + a * d), 0, 1)

        # Update x and compute differences
        x_new = x + alpha * d
        grad_new = grad_f(x_new)
        s = x_new - x
        y = grad_new - grad

        # Curvature condition check
        ys = np.dot(y, s)
        if ys > 1e-10:
            rho = 1.0 / ys
            I = np.eye(n)
            H = (I - rho * np.outer(s, y)) @ H @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)


        x, grad = x_new, grad_new

    return x, history


Example Usage

In [ ]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

# Define the gradient
def grad_rosenbrock(x):
    grad = np.zeros_like(x)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[1] = 200 * (x[1] - x[0]**2)
    return grad

# Initial guess
x0 = [-1.2, 1.0]


x_opt, history = quasi_newton_bfgs(rosenbrock, grad_rosenbrock, x0)


print("Quasi-Newton (BFGS) Method Results:")
print(f"Optimal solution: {x_opt}")
print(f"Function value at optimum: {rosenbrock(x_opt)}")
print(f"Number of iterations: {len(history['iterations'])}")


Quasi-Newton (BFGS) Method Results:
Optimal solution: [1. 1.]
Function value at optimum: 1.0095170856086831e-18
Number of iterations: 18


# Part 3:solve the benchmark problems:

In [ ]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

def grad_rosenbrock(x):
    grad = np.zeros_like(x)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[1] = 200 * (x[1] - x[0]**2)
    return grad

def hessian_rosenbrock(x):
    hessian = np.zeros((len(x), len(x)))
    hessian[0, 0] = 1200 * x[0]**2 - 400 * x[1] + 2
    hessian[0, 1] = -400 * x[0]
    hessian[1, 0] = -400 * x[0]
    hessian[1, 1] = 200
    return hessian

# Rosenbrock Function
x0_rosen = [-1.2, 1.0]
rosen_cg, history_cg = fletcher_reeves_cg(rosenbrock, grad_rosenbrock, x0_rosen)
rosen_marquardt, history_marquardt = marquardt_method(rosenbrock, grad_rosenbrock, hessian_rosenbrock, x0_rosen)
rosen_bfgs, history_bfgs = quasi_newton_bfgs(rosenbrock, grad_rosenbrock, x0_rosen)

print("Fletcher-Reeves CG Results:")
print("Optimal solution:", rosen_cg)
print("Function value:", rosenbrock(rosen_cg))

print("\nMarquardt Method Results:")
print("Optimal solution:", rosen_marquardt)
print("Function value:", rosenbrock(rosen_marquardt))

print("\nQuasi-Newton (BFGS) Results:")
print("Optimal solution:", rosen_bfgs)
print("Function value:", rosenbrock(rosen_bfgs))

Fletcher-Reeves CG Results:
Optimal solution: [1.00000291 1.00000585]
Function value: 8.563102755875322e-12

Marquardt Method Results:
Optimal solution: [0.99999998 0.99999996]
Function value: 4.866187419097994e-16

Quasi-Newton (BFGS) Results:
Optimal solution: [1. 1.]
Function value: 1.0095170856086831e-18


In [ ]:
import time

def powell_quartic(x):
    return (x[0] + 10 * x[1])**2 + 5 * (x[2] - x[3])**2 + (x[1] - 2 * x[2])**4 + 10 * (x[0] - x[3])**4

def grad_powell_quartic(x):
    grad = np.zeros_like(x)
    grad[0] = 2 * (x[0] + 10 * x[1]) + 40 * (x[0] - x[3])**3
    grad[1] = 20 * (x[0] + 10 * x[1]) + 4 * (x[1] - 2 * x[2])**3
    grad[2] = 10 * (x[2] - x[3]) - 8 * (x[1] - 2 * x[2])**3
    grad[3] = -10 * (x[2] - x[3]) - 40 * (x[0] - x[3])**3
    return grad

results = []


x0_rosen = [-1.2, 1.0]
for method, func in zip([
        fletcher_reeves_cg,
        marquardt_method,
        quasi_newton_bfgs
    ],
    [
        "Fletcher-Reeves CG",
        "Marquardt Method",
        "Quasi-Newton (BFGS)"
    ]):
    start_time = time.time()
    if method == marquardt_method:
        x_opt, history = method(rosenbrock, grad_rosenbrock, hessian_rosenbrock, x0_rosen)
    else:
        x_opt, history = method(rosenbrock, grad_rosenbrock, x0_rosen)
    end_time = time.time()
    results.append({
        "Function": "Rosenbrock",
        "Method": func,
        "Iterations": len(history["iterations"]),
        "Optimal Solution": x_opt,
        "Optimal Value": rosenbrock(x_opt),
        "CPU Time": end_time - start_time
    })

x0_powell = [3.0, -1.0, 0.0, 1.0]
for method, func in zip([
        fletcher_reeves_cg,
        marquardt_method,
        quasi_newton_bfgs
    ],
    [
        "Fletcher-Reeves CG",
        "Marquardt Method",
        "Quasi-Newton (BFGS)"
    ]):
    start_time = time.time()
    if method == marquardt_method:
        x_opt, history = method(powell_quartic, grad_powell_quartic, hessian_rosenbrock, x0_powell)
    else:
        x_opt, history = method(powell_quartic, grad_powell_quartic, x0_powell)
    end_time = time.time()
    results.append({
        "Function": "Powell's Quartic",
        "Method": func,
        "Iterations": len(history["iterations"]),
        "Optimal Solution": x_opt,
        "Optimal Value": powell_quartic(x_opt),
        "CPU Time": end_time - start_time
    })

for result in results:
    print(result)

{'Function': 'Rosenbrock', 'Method': 'Fletcher-Reeves CG', 'Iterations': 41, 'Optimal Solution': array([1.00000291, 1.00000585]), 'Optimal Value': 8.563102755875322e-12, 'CPU Time': 0.011466741561889648}
{'Function': 'Rosenbrock', 'Method': 'Marquardt Method', 'Iterations': 41, 'Optimal Solution': array([0.99999998, 0.99999996]), 'Optimal Value': 4.866187419097994e-16, 'CPU Time': 0.0016703605651855469}
{'Function': 'Rosenbrock', 'Method': 'Quasi-Newton (BFGS)', 'Iterations': 18, 'Optimal Solution': array([1., 1.]), 'Optimal Value': 1.0095170856086831e-18, 'CPU Time': 0.0038330554962158203}
{'Function': "Powell's Quartic", 'Method': 'Fletcher-Reeves CG', 'Iterations': 1000, 'Optimal Solution': array([ 0.00456979, -0.00045688,  0.00227233,  0.00227198]), 'Optimal Value': 9.061386050412977e-10, 'CPU Time': 0.24268436431884766}
{'Function': "Powell's Quartic", 'Method': 'Marquardt Method', 'Iterations': 100, 'Optimal Solution': array([ 2.98535363, -0.27160518,  0.53352286,  2.23337052]), 

Algorithm iterations

In [ ]:
import pandas as pd

def print_iterations(history, method_name, function_name):
    """
    Prints all iterations for a given optimization history.
    """
    df = pd.DataFrame({
        "Iteration": history["iterations"],
        "Function Value": history["function_values"],
        "Gradient Norm": history["gradient_norms"],
    })
    print(f"--- Iterations for {function_name} using {method_name} ---")
    print(df.to_string(index=False))
    print("\n")



x0_rosen = [-1.2, 1.0]
rosen_cg, history_cg = fletcher_reeves_cg(rosenbrock, grad_rosenbrock, x0_rosen)
rosen_marquardt, history_marquardt = marquardt_method(rosenbrock, grad_rosenbrock, hessian_rosenbrock, x0_rosen)
rosen_bfgs, history_bfgs = quasi_newton_bfgs(rosenbrock, grad_rosenbrock, x0_rosen)

print_iterations(history_cg, "Fletcher-Reeves CG", "Rosenbrock")
print_iterations(history_marquardt, "Marquardt Method", "Rosenbrock")
print_iterations(history_bfgs, "Quasi-Newton (BFGS)", "Rosenbrock")

x0_powell = [3.0, -1.0, 0.0, 1.0]
powell_cg, history_powell_cg = fletcher_reeves_cg(powell_quartic, grad_powell_quartic, x0_powell)
powell_bfgs, history_powell_bfgs = quasi_newton_bfgs(powell_quartic, grad_powell_quartic, x0_powell)

print_iterations(history_powell_cg, "Fletcher-Reeves CG", "Powell's Quartic")
print_iterations(history_powell_bfgs, "Quasi-Newton (BFGS)", "Powell's Quartic")


--- Iterations for Rosenbrock using Fletcher-Reeves CG ---
 Iteration  Function Value  Gradient Norm
         0    2.420000e+01     232.867688
         1    1.947040e-01       0.560662
         2    1.945893e-01       0.336626
         3    1.291814e-01       8.383414
         4    8.787214e-02       9.820585
         5    5.660007e-02       9.320946
         6    3.058880e-02       7.585581
         7    9.250582e-03       4.301214
         8    1.167973e-03       0.536809
         9    1.043169e-03       0.056987
        10    1.041359e-03       0.031245
        11    1.030119e-03       0.158075
        12    2.962355e-04       0.753879
        13    3.399322e-05       0.127403
        14    2.814352e-05       0.019463
        15    2.800165e-05       0.005118
        16    2.797457e-05       0.007156
        17    2.744717e-05       0.037893
        18    1.576637e-05       0.134747
        19    1.598075e-06       0.049291
        20    7.045557e-07       0.007945
        21    6.8

Final Comparing

In [ ]:

import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)


           Function               Method  Iterations  \
0        Rosenbrock   Fletcher-Reeves CG          41   
1        Rosenbrock     Marquardt Method          41   
2        Rosenbrock  Quasi-Newton (BFGS)          18   
3  Powell's Quartic   Fletcher-Reeves CG        1000   
4  Powell's Quartic     Marquardt Method         100   
5  Powell's Quartic  Quasi-Newton (BFGS)          32   

                                    Optimal Solution  Optimal Value  CPU Time  
0             [1.0000029089194933, 1.00000584967356]   8.563103e-12  0.018179  
1           [0.9999999779435205, 0.9999999558509232]   4.866187e-16  0.003407  
2           [0.9999999991350941, 0.9999999982190555]   1.009517e-18  0.007212  
3  [0.004569793565841066, -0.00045688192804389327...   9.061386e-10  0.341768  
4  [2.985353632890308, -0.271605179790214, 0.5335...   2.092881e+01  0.004395  
5  [-0.0027892681745203263, 0.000278916325489459,...   1.166766e-10  0.008371  
